# library

In [4]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc, rcParams
import seaborn as sns
from fbprophet import Prophet
import datetime
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import warnings
warnings.filterwarnings('ignore')

plt.style.use('ggplot')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
rcParams['figure.figsize'] = (16, 8)
rc('font', family='AppleGothic')

In [2]:
base_path = './open_data'
sub_path = './submission'

In [3]:
df = pd.read_csv(os.path.join(base_path, 'train.csv'), engine = 'python', encoding = 'cp949')
test = pd.read_csv(os.path.join(base_path, 'submission.csv'), engine = 'python', encoding = 'cp949')

# Naive predict

In [5]:
# 날짜 추출, submission 추출
submission_df = pd.DataFrame()
submission_df['DateTime'] = pd.date_range('2020-11-09','2021-01-08')
submission_df['DateTime'] = submission_df['DateTime'].apply(
            lambda x: datetime.datetime.strftime(x,'%Y-%m-%d')
        )

df2 = df[['DateTime','사용자']]
df2.columns = ['ds','y']

model = Prophet()
model.fit(df2)

# 시간 단위 예측 -> 평균내서 제출해볼 것(오차보정도 해야함)
future = model.make_future_dataframe(periods = 1464, freq = 'H')
fcst = model.predict(future)
pred_df = fcst.iloc[19008:]

INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [27]:
temp = pd.DataFrame()
temp['DateTime'] = pred_df['ds']
temp.index = range(len(temp))

temp['DateTime'] = \
        temp['DateTime'].apply(
            lambda x: datetime.datetime.strftime(x,'%Y-%m-%d')
    )


In [34]:
temp2 = pd.concat([temp,pd.DataFrame(df[df['DateTime'] >= '2020-10-09 00:00:00'].iloc[:,1:].values)],axis = 1)
temp2.columns = ['DateTime'] + df.columns[1:].tolist()

In [38]:
temp3 = temp2.groupby('DateTime').sum().reset_index()

In [41]:
time_str = datetime.datetime.strftime(
    datetime.datetime.today(),
    '%Y%m%d_%H%M%S'
)
temp3.to_csv(
    os.path.join(sub_path, f'submission_naive_{time_str}.csv'),
    index = False,
    encoding = 'cp949'
)

# weight predict

In [ ]:
# 가중치를 주면서 예측


# MA

# holt linear